In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import jensenshannon
from pathlib import Path
import sys
import matplotlib.pyplot as plt


In [ ]:
BASE_PATH = Path.cwd().parent
TESTING_FILES_BASE_PATH = BASE_PATH/"testing"/"files"
sys.path.append(str(BASE_PATH))

In [ ]:
from model.main import EMOTIONS

def plot_beliefs(belief_user, belief_character, belief_user_character):
    x = np.arange(len(EMOTIONS))
    width = 0.25  # Slightly smaller width to fit 3 bars

    label1 = "System's belief about User"
    label2 = "System's belief about other character"
    label3 = "System's belief about User's belief about other character"

    fig, ax = plt.subplots()
    bars1 = ax.bar(x - width, belief_user, width, label=label1)
    bars2 = ax.bar(x, belief_character, width, label=label2)
    bars3 = ax.bar(x + width, belief_user_character, width, label=label3)

    ax.set_xlabel('Mental States')
    ax.set_ylabel('Probability')
    ax.set_title('Belief Distribution')
    ax.set_xticks(x)
    ax.set_xticklabels(EMOTIONS)
    ax.legend()

    plt.ylim(0, 1)
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
# Entropy: using log base 2
def entropy(dist):
    dist = np.array(dist)
    dist = dist[dist > 0]  # Avoid log(0)
    return -np.sum(dist * np.log2(dist))

In [ ]:
# Takes data from neg_correlation_emotions, pos_correlation_emotions and uncorrelated_emotions distributions
# Generates stats on distribution divergence (Jensen-Shannon divergence) 
class TestCaseDivergence():
    # Read files and generate stats
    def __init__(self, folder_path_dists, file_path_messages):
        with open(file_path_messages, encoding="utf-8") as f_explicit:
            self.messages = f_explicit.read().split("\n")

        self.num_files = len(self.messages)
        self.System_beliefs_user = [] # dist 1: System's belief of User
        self.System_beliefs_character = [] # dist 2: System's belief of other character
        self.System_beliefs_users_belief_character = [] # dist 3: System's belief of User's belief of other character
        self.jsds_1_2 = []
        self.jsds_1_3 = []
        self.jsds_2_3 = []

        for i in range(self.num_files):
            df = pd.read_csv(folder_path_dists/("message_"+str(i+1)+"_belief_dist.csv"))
            self.System_beliefs_user.append(df["Prob_System_belief_User"])
            self.System_beliefs_character.append(df["Prob_System_belief_Character"])
            self.System_beliefs_users_belief_character.append(df["Prob_System_belief_User_belief_Character"])

            # Jensen-Shannon Divergences
            jsd_1_2 = jensenshannon(df["Prob_System_belief_User"], df["Prob_System_belief_Character"])
            self.jsds_1_2.append(jsd_1_2)

            jsd_1_3 = jensenshannon(df["Prob_System_belief_User"], df["Prob_System_belief_User_belief_Character"])
            self.jsds_1_3.append(jsd_1_3)

            jsd_2_3 = jensenshannon(df["Prob_System_belief_Character"], df["Prob_System_belief_User_belief_Character"])
            self.jsds_2_3.append(jsd_2_3)
            
    def display_stats(self):
        for i in range(self.num_files):
            print("\n------------------------------------------------------------------" \
            "\n"+str(i+1)+"-User message:", self.messages[i])
            plot_beliefs(self.System_beliefs_user[i], self.System_beliefs_character[i], self.System_beliefs_users_belief_character[i])
            print("JSD - between System's belief of User & System's belief of other character: ", self.jsds_1_2[i])
            
            print("JSD - between System's belief of User & System's belief of User's belief of other character: ", self.jsds_1_3[i])
            
            print("JSD - between System's belief of Other character & System's belief of User's belief of other character: ", self.jsds_2_3[i])
            

In [ ]:
# Takes data from user_emotions_explicit_character_emotions_explicit,
#                 user_emotions_explicit_character_emotions_implicit,
#                 user_emotions_implicit_character_emotions_explicit,
#                 user_emotions_implicit_character_emotions_implicit,
# Generates stats on entropy
class TestCaseEntropy():
    # Read files and generate stats
    def __init__(self, folder_path_dists, file_path_messages):
        with open(file_path_messages, encoding="utf-8") as f_explicit:
            self.messages = f_explicit.read().split("\n")

        self.num_files = len(self.messages)
        self.System_beliefs_user = [] # dist 1: System's belief of User
        self.System_beliefs_character = [] # dist 2: System's belief of other character
        self.System_beliefs_users_belief_character = [] # dist 3: System's belief of User's belief of other character
        self.entropies_1 = [] # Entropy of dist 1
        self.entropies_2 = [] # Entropy of dist 2
        self.entropies_3 = [] # Entropy of dist 3

        for i in range(self.num_files):
            df = pd.read_csv(folder_path_dists/("message_"+str(i+1)+"_belief_dist.csv"))
            self.System_beliefs_user.append(df["Prob_System_belief_User"])
            self.System_beliefs_character.append(df["Prob_System_belief_Character"])
            self.System_beliefs_users_belief_character.append(df["Prob_System_belief_User_belief_Character"])

            self.entropies_1.append(entropy(df["Prob_System_belief_User"]))
            self.entropies_2.append(entropy(df["Prob_System_belief_Character"]))
            self.entropies_3.append(entropy(df["Prob_System_belief_User_belief_Character"]))


    def display_stats(self):
        for i in range(self.num_files):
            print("\n------------------------------------------------------------------" \
            "\n"+str(i+1)+"-User message:", self.messages[i])
            plot_beliefs(self.System_beliefs_user[i], self.System_beliefs_character[i], self.System_beliefs_users_belief_character[i])
            print("Entropy - System's belief of User: ", self.entropies_1[i])
            print("Entropy - System's belief of other character: ", self.entropies_2[i])
            print("Entropy - System's belief of User's belief of other character: ", self.entropies_3[i])
            

Get and display all results individually

In [ ]:
DISTRIBUTIONS_BASE_PATH = TESTING_FILES_BASE_PATH/"distributions"
MESSAGES_BASE_PATH = TESTING_FILES_BASE_PATH/"messages"

# Divergence test cases
uncorrelated_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"uncorrelated_emotions", MESSAGES_BASE_PATH/"uncorrelated_emotions.txt")
pos_correlated_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"pos_correlated_emotions", MESSAGES_BASE_PATH/"pos_correlated_emotions.txt")
neg_correlated_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"neg_correlated_emotions", MESSAGES_BASE_PATH/"neg_correlated_emotions.txt")

# Display individual message stats
print("JENSON-SHANNON DIVERGENCE STATS \n\n")

print("Uncorrelated: \n")
uncorrelated_test_case.display_stats()
print("Positively correlated: \n")
#pos_correlated_test_case.display_stats()
print("Negatively correlated: \n")
#neg_correlated_test_case.display_stats()


In [ ]:

# Entropy test cases
user_exp_character_exp_test_case = TestCaseEntropy(DISTRIBUTIONS_BASE_PATH/"user_emotions_explicit_character_emotions_explicit", MESSAGES_BASE_PATH/"user_emotions_explicit_character_emotions_explicit.txt")
user_exp_character_imp_test_case = TestCaseEntropy(DISTRIBUTIONS_BASE_PATH/"user_emotions_explicit_character_emotions_implicit", MESSAGES_BASE_PATH/"user_emotions_explicit_character_emotions_implicit.txt")
user_imp_character_exp_test_case = TestCaseEntropy(DISTRIBUTIONS_BASE_PATH/"user_emotions_implicit_character_emotions_explicit", MESSAGES_BASE_PATH/"user_emotions_implicit_character_emotions_explicit.txt")
user_imp_character_imp_test_case = TestCaseEntropy(DISTRIBUTIONS_BASE_PATH/"user_emotions_implicit_character_emotions_implicit", MESSAGES_BASE_PATH/"user_emotions_implicit_character_emotions_implicit.txt")

# Display individual message stats
print("ENTROPY STATS \n\n")

print("User emotions explicit, other character emotions explicit: \n")
user_exp_character_exp_test_case.display_stats()

print("User emotions explicit, other character emotions implicit: \n")
user_exp_character_imp_test_case.display_stats()

print("User emotions implicit, other character emotions explicit: \n")
user_imp_character_exp_test_case.display_stats()

print("User emotions implicit, other character emotions implicit: \n")
user_imp_character_imp_test_case.display_stats()

High level overview

In [ ]:
def show_aggregated_results_correlation(uncorrelated_test_case, pos_correlated_test_case, neg_correlated_test_case):
    labels = [
        'Character & User emotions uncorrelated', 'Character & User emotions positively correlated', 'Character & User emotions negatively correlated'
    ]

    # Pearson Correlation - between System's belief of User & System's belief of other character
    plt.boxplot([
        uncorrelated_test_case.jsds_1_2,
        pos_correlated_test_case.jsds_1_2,
        neg_correlated_test_case.jsds_1_2,
    ], labels=labels)
    plt.ylabel("JS Divergence")
    plt.title("JSD - between System's belief of User & System's belief of other character")
    plt.ylim(0, 1)
    plt.xticks(rotation=90)
    plt.show()

    # Spearman Correlation - between System's belief of User & System's belief of other character
    plt.boxplot([
        uncorrelated_test_case.jsds_1_2,
        pos_correlated_test_case.jsds_1_2,
        neg_correlated_test_case.jsds_1_2,
    ], labels=labels)
    plt.ylabel("JS Divergence")
    plt.title("JSD - between System's belief of User & System's belief of other character")
    plt.ylim(0, 1)
    plt.xticks(rotation=90)
    plt.show()

    # Pearson Correlation - between System's belief of User & System's belief of User's belief of other character
    plt.boxplot([
        uncorrelated_test_case.jsds_1_3,
        pos_correlated_test_case.jsds_1_3,
        neg_correlated_test_case.jsds_1_3,
    ], labels=labels)
    plt.ylabel("JS Divergence")
    plt.title("JSD - between System's belief of User & System's belief of User's belief of other character")
    plt.ylim(0, 1)
    plt.xticks(rotation=90)
    plt.show()


show_aggregated_results_correlation(uncorrelated_test_case, pos_correlated_test_case, neg_correlated_test_case)


In [ ]:
def show_aggregated_results_enotropy(user_exp_character_exp_test_case, user_exp_character_imp_test_case, user_imp_character_exp_test_case, user_imp_character_imp_test_case):
    labels = [
        "User explicit, other explicit",
        "User explicit, other implicit",
        "User implicit, other explicit",
        "User implicit, other implicit"
    ]

    # Entropy - System's belief of User
    plt.boxplot([
        user_exp_character_exp_test_case.entropies_1,
        user_exp_character_imp_test_case.entropies_1,
        user_imp_character_exp_test_case.entropies_1,
        user_imp_character_imp_test_case.entropies_1,
    ], labels=labels)
    plt.ylabel("Entropy")
    plt.title("Entropy - System's belief of User")
    plt.ylim(0, 10)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Entropy - System's belief of other character
    plt.boxplot([
        user_exp_character_exp_test_case.entropies_2,
        user_exp_character_imp_test_case.entropies_2,
        user_imp_character_exp_test_case.entropies_2,
        user_imp_character_imp_test_case.entropies_2,
    ], labels=labels)
    plt.ylabel("Entropy")
    plt.title("Entropy - System's belief of other character")
    plt.ylim(0, 10)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Entropy - System's belief of User's belief of other character
    plt.boxplot([
        user_exp_character_exp_test_case.entropies_3,
        user_exp_character_imp_test_case.entropies_3,
        user_imp_character_exp_test_case.entropies_3,
        user_imp_character_imp_test_case.entropies_3,
    ], labels=labels)
    plt.ylabel("Entropy")
    plt.title("Entropy - System's belief of User's belief of other character")
    plt.ylim(0, 10)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

show_aggregated_results_enotropy(user_exp_character_exp_test_case, user_exp_character_imp_test_case, user_imp_character_exp_test_case, user_imp_character_imp_test_case)